In [74]:
import torch
import torch.nn as nn
from math import ceil
import torch.nn.functional as F

import torchvision as tv

import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from tqdm import tqdm

from evaluations.kaggle_2020 import global_average_precision_score
from sklearn.preprocessing import LabelEncoder

from torch.cuda.amp import autocast, GradScaler

In [75]:
base_model = [
    #expand_ratio, channels, repeats, stride, kernel_size
    [1,16,1,1,3],
    [6,24,2,2,3],
    [6,40,2,2,5],
    [6,80,3,2,3],
    [6,112,3,1,5],
    [6,320,1,1,3]
]

In [76]:
phi_values ={
    #tuple of: (phi_value, resolution, drop_rate)
    "b0": (0, 224, 0.2), #alpha, beta, gamma, depth = alpha**phi
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

In [77]:
class CNNBlock(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size, stride, padding, groups=1):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            groups=groups,
            bias=False,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU()
    def forward(self, x):
        return self.silu(self.bn(self.cnn(x)))

class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim,in_channels,1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        return x * self.se(x)

class InvertedResidualBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        expand_ratio,
        reduction=4, # squeeze excitation
        survival_prob=0.8, # for stochastic depth
    ):
        super(InvertedResidualBlock, self).__init__()
        self.survival_prob=0.8
        self.use_residual = in_channels == out_channels and stride ==1
        hidden_dim = in_channels * expand_ratio
        self.expand = in_channels != hidden_dim
        reduced_dim = int(in_channels/reduction)

        if self.expand:
            self.expand_conv = CNNBlock(
                in_channels, hidden_dim, kernel_size=3, stride=1, padding=1,
            )

        self.conv = nn.Sequential(
            CNNBlock(
                hidden_dim, hidden_dim, kernel_size, stride, padding, groups=hidden_dim,
            ),
            SqueezeExcitation(hidden_dim, reduced_dim),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

    def stochastic_depth(self,x):
        if not self.training:
            return x
        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.survival_prob
        return torch.div(x, self.survival_prob) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs

        if self.use_residual:
            return self.stochastic_depth(self.conv(x)) + inputs
        else:
            return self.conv(x)
class EfficientNet(nn.Module):
    def __init__(self, version, num_classes):
        super(EfficientNet, self).__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(version)
        last_channels = ceil(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.features = self.create_features(width_factor, depth_factor, last_channels)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, num_classes),
        )

    def calculate_factors(self, version, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[version]
        depth_factor = alpha ** phi
        width_factor = beta ** phi
        return width_factor, depth_factor, drop_rate

    def create_features(self, width_factor, depth_factor, last_channels):
        channels = int(32 * width_factor)
        features = [CNNBlock(3, channels, 3, stride=2, padding=1)]
        in_channels = channels
        for expand_ratio, channels, repeats, stride, kernel_size in base_model:
            out_channels = 4 * ceil(int(channels*width_factor)/4)
            layers_repeats = ceil(repeats * depth_factor)
            for layer in range(layers_repeats):
                features.append(
                    InvertedResidualBlock(
                        in_channels,
                        out_channels,
                        expand_ratio=expand_ratio,
                        stride = stride if layer == 0 else 1,
                        kernel_size=kernel_size,
                        padding=kernel_size//2,
                    )
                )
                in_channels = out_channels
        features.append(
            CNNBlock(in_channels, last_channels, kernel_size=1, stride=1, padding=0)
        )

        return nn.Sequential(*features)

    def forward(self,x):
        x = self.pool(self.features(x))
        return self.classifier(x.view(x.shape[0],-1))

In [78]:
class DatasetToClass(torch.utils.data.Dataset):
    def __init__(self, path_dirs):
        super().__init__()
        
        self.path_dirs = []
        for i in path_dirs:
            self.path_dirs.append(i)
        
        self.class_borders=[]
        self.class_borders.append(0)
        #print(sorted(os.listdir(path_dirs[0])))
        self.dir_list=[]
        for i in range(1, len(path_dirs)+1):
            self.dir_list.append(sorted(os.listdir(path_dirs[i-1])))
            self.class_borders.append(len(self.dir_list[i-1])+ self.class_borders[i-1])
        #print(self.class_borders)
    
    def __len__(self):
        length = 0
        for i in self.dir_list:
            length += len(i)
        return length
    
    def __getitem__(self,idx):
        class_id=0
        for i in range(1,11):
            if idx >= self.class_borders[i-1] and idx < self.class_borders[i]:
                class_id = i-1
                break
        img_path = os.path.join(self.path_dirs[class_id], self.dir_list[class_id][self.class_borders[class_id]-idx])

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        img=img.astype(np.float32)
        img = img/255.0
        img = cv2.resize(img,(128,128), interpolation = cv2.INTER_AREA)
        img = img.transpose((2,0,1))
        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)
        #return img
        return {'img': t_img, 'label':t_class_id}        

In [79]:
train_ds = DatasetToClass([
    '/home/andrey/GraduateWork/IMG_CLASSES/1. Eczema 1677',
    '/home/andrey/GraduateWork/IMG_CLASSES/2. Melanoma 15.75k',
    '/home/andrey/GraduateWork/IMG_CLASSES/3. Atopic Dermatitis - 1.25k',
    '/home/andrey/GraduateWork/IMG_CLASSES/4. Basal Cell Carcinoma (BCC) 3323',
    '/home/andrey/GraduateWork/IMG_CLASSES/5. Melanocytic Nevi (NV) - 7970',
    '/home/andrey/GraduateWork/IMG_CLASSES/6. Benign Keratosis-like Lesions (BKL) 2624',
    '/home/andrey/GraduateWork/IMG_CLASSES/7. Psoriasis pictures Lichen Planus and related diseases - 2k',
    '/home/andrey/GraduateWork/IMG_CLASSES/8. Seborrheic Keratoses and other Benign Tumors - 1.8k',
    '/home/andrey/GraduateWork/IMG_CLASSES/9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k',
    '/home/andrey/GraduateWork/IMG_CLASSES/10. Warts Molluscum and other Viral Infections - 2103'
])

test_ds = DatasetToClass([
    '/home/andrey/GraduateWork/IMG_CLASSES/1. TEczema 1677',
    '/home/andrey/GraduateWork/IMG_CLASSES/2. TMelanoma 15.75k',
    '/home/andrey/GraduateWork/IMG_CLASSES/3. TAtopic Dermatitis - 1.25k',
    '/home/andrey/GraduateWork/IMG_CLASSES/4. TBasal Cell Carcinoma (BCC) 3323',
    '/home/andrey/GraduateWork/IMG_CLASSES/5. TMelanocytic Nevi (NV) - 7970',
    '/home/andrey/GraduateWork/IMG_CLASSES/6. TBenign Keratosis-like Lesions (BKL) 2624',
    '/home/andrey/GraduateWork/IMG_CLASSES/7. TPsoriasis pictures Lichen Planus and related diseases - 2k',
    '/home/andrey/GraduateWork/IMG_CLASSES/8. TSeborrheic Keratoses and other Benign Tumors - 1.8k',
    '/home/andrey/GraduateWork/IMG_CLASSES/9. TTinea Ringworm Candidiasis and other Fungal Infections - 1.7k',
    '/home/andrey/GraduateWork/IMG_CLASSES/10. TWarts Molluscum and other Viral Infections - 2103'
])

In [89]:
batch_size = 96
train_loader = torch.utils.data.DataLoader(train_ds, shuffle = True, batch_size = batch_size, num_workers=1, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_ds, shuffle = True, batch_size = batch_size, num_workers=1, drop_last=False)

In [98]:
device = "cuda" if torch.cuda.is_available() else "cpu"
version = "b4"
num_classes = 10
model = EfficientNet(
    version=version,
    num_classes=num_classes,
).to(device)


In [99]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9,0.999), eps=1e-3, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=1e-6)

In [100]:
def GAP( y_true,y_pred):
    return global_average_precision_score(y_true, y_pred)

In [101]:
def accuracy(pred, label):
    answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1) 
    return answer.mean()

In [102]:
use_amp = True
scaler = torch.cuda.amp.GradScaler()

In [103]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

In [104]:
n_epochs = 15
loss_list = []
#activation = nn.Softmax(dim=1)
loss_list=[]
gap_score_list=[]
model.train()
for epochs in range(n_epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar:=tqdm(train_loader)):        
        img=sample['img']
        label=sample['label']  
        
        optimizer.zero_grad()
        
        label = F.one_hot(label, 10).float()
        img=img.to(device)
        label = label.to(device)

        with autocast(use_amp):
            pred = model(img)
            loss = loss_fn(pred, label)  
        
        #conf_scores, pred_labels = torch.max(yhat.detach(), dim=1)
        
        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item

        scaler.step(optimizer)
        scaler.update()

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

        
    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
    print(loss_val/len(train_loader))
    print(acc_val/len(train_loader))

  0%|                                                                                           | 0/254 [01:04<?, ?it/s]Traceback (most recent call last):

  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [97]:
loss_val = 0
acc_val = 0
for sample in (pbar := tqdm(test_loader)):
    with torch.no_grad():
        img, label = sample['img'], sample['label']

        label = F.one_hot(label, 10).float()
        img=img.to(device)
        label = label.to(device)
        pred = model(img)

        loss = loss_fn(pred, label)
        loss_item = loss.item()
        loss_val += loss_item

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

    pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
print(loss_val/len(test_loader))
print(acc_val/len(test_loader))

  0%|                                                                                            | 0/29 [00:00<?, ?it/s]/tmp/ipykernel_491/156703249.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
loss: 1.35449	accuracy: 0.533: 100%|████████████████████████████████████████████████████| 29/29 [00:39<00:00,  1.38s/it]

0.9802288022534601
0.6125000000000003
